In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
import cvxpy as cp
from plotly.subplots import make_subplots
from croniter import croniter

In [2]:
instruments = [
    'META',
    'TSLA',
    'NVDA',
    'AAPL',
    'EWY',
    'DXJ',
    'BRK-B',
    'SPY',
    'QQQ'
]

end_date = get_today()
start_date = pd.to_datetime(datetime(2022, 1, 3))
max_leverage = 1
vol_target = 0.25

# Notes

- 2024-08-28: Tested expected return prediction with RandomForest, XGB, LGBM. None of them outperforms the SMA Model. LGBM is able to achieve a similar performance as SMA model and generally train faster.</br>

    | Measure                | ^SPX      | MVO - SMA1 | MVO - RandomForest1 | MVO - XGB1 | MVO - LGBM1 |
    |------------------------|-----------|------------|---------------------|------------|-------------|
    | Cumulative Return      | 1.173503  | 2.072994   | 1.662573            | 1.471717   | 2.023378    |
    | Annualized Return      | 0.076775  | 0.301537   | 0.221185            | 0.176047   | 0.296252    |
    | Annualized Volatility  | 0.180081  | 0.226417   | 0.240133            | 0.243953   | 0.243120    |
    | Annualized Sharpe Ratio| 0.240438  | 1.183923   | 0.781685            | 0.584416   | 1.080844    |
    | Maximum Drawdown       | -0.254251 | -0.271732  | -0.323658           | -0.380737  | -0.296535   |

    Models are shared the same hyperparameters: lookback (train days) = 10 days and gamma=10, hhi=0.2


In [5]:
portfolio = PortfolioStandard(
    capital=169336.48958837058,
    name='MVOPortfolio',        
    rebalance_iter=RebalancerIter('0 0 * * Fri', 2),        
    tc_model=TransactionCostFutu(), 
    systems=[        
        #TradingSubSystemSingle(vol_target=vol_target, instruments=instruments,  strategy=[MeanVarianceOptRetLookAheadBias(gamma=10, hhi=0.2, lookback=60, confidence=1, lookahead=10)], max_leverage=max_leverage, offset=60),
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments,  strategy=[MeanVarianceOpt(gamma=10, hhi=0.2, lookback=10, confidence=1)], max_leverage=max_leverage, offset=60),
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments,  strategy=[MeanVarianceOptRandomForest(gamma=10, hhi=0.2, lookback=10, confidence=1, train_days=10)], max_leverage=max_leverage, offset=60),
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments,  strategy=[MeanVarianceOptXGB(gamma=10, hhi=0.2, lookback=10, confidence=1, train_days=10)], max_leverage=max_leverage, offset=60),
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments,  strategy=[MeanVarianceOptLGBM(gamma=10, hhi=0.2, lookback=10, confidence=1, train_days=10)], max_leverage=max_leverage, offset=60),
    ]
)

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize()
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

2024-08-28 01:53:37,787 - TradingSubSystemSingle [MVO - SMA1] - INFO - Generating position for strategy MVO - SMA1 between 2021-12-31 and 2024-08-27......
100%|██████████| 693/693 [00:04<00:00, 145.22it/s]
2024-08-28 01:53:42,576 - TradingSubSystemSingle [MVO - SMA1] - INFO - Volatility Target = 25.0% | Price Volatility = 25.9% | Last Scale Factor = 0.97
2024-08-28 01:53:42,577 - TradingSubSystemSingle [MVO - RandomForest1] - INFO - Generating position for strategy MVO - RandomForest1 between 2021-12-31 and 2024-08-27......
100%|██████████| 693/693 [04:22<00:00,  2.64it/s]
2024-08-28 01:58:05,596 - TradingSubSystemSingle [MVO - RandomForest1] - INFO - Volatility Target = 25.0% | Price Volatility = 23.0% | Last Scale Factor = 1.08
2024-08-28 01:58:05,603 - TradingSubSystemSingle [MVO - XGB1] - INFO - Generating position for strategy MVO - XGB1 between 2021-12-31 and 2024-08-27......
100%|██████████| 693/693 [06:10<00:00,  1.87it/s]
2024-08-28 02:04:16,182 - TradingSubSystemSingle [MVO -

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Rebalanced Portfolio,Optimized Portfolio,^SPX,MVO - SMA1,MVO - RandomForest1,MVO - XGB1,MVO - LGBM1
Measure,,,,,,,
Cumulative Return,1.733662,1.728899,1.173503,2.072994,1.662573,1.471717,2.023378
Annualized Return,0.236295,0.235564,0.076775,0.301537,0.221185,0.176047,0.296252
Annualized Volatility,0.237206,0.238303,0.180081,0.226417,0.240133,0.243953,0.243120
Annualized Sharpe Ratio,0.855029,0.848028,0.240438,1.183923,0.781685,0.584416,1.080844
Maximum Drawdown,-0.348532,-0.332489,-0.254251,-0.271732,-0.323658,-0.380737,-0.296535


In [4]:
portfolio.get_position_for_trade()

2024-08-27 02:12:11,586 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $169,336
2024-08-27 02:12:11,587 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $169,336


""


# Execute the Portfolio

In [168]:
# executor = ExecutorFutu(is_test=True)
# executor.set_portfolio(portfolio)
# orders = executor.execute(px_interval='5m')
# orders